# 对话式 RAG

[官方教程](http://www.aidoczh.com/langchain/v0.2/docs/tutorials/qa_chat_history/)

In [1]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain     # ？
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.output_parsers import StrOutputParser                       # ？
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough                        # ？
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI

# llm = OpenAI(
#     base_url = 'http://localhost:8000/v1',
#     api_key = 'token-qwen2',
#     model = 'Qwen2'
# )

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4-airx",
    openai_api_key="",    # 加入 API-KEY
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

## Step1. 构造 链

In [2]:
data_path = './data'
loader = DirectoryLoader(data_path, glob="tencent2.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embed_model_path = '/home/yhchen/huggingface_model/BAAI/bge-m3'
BgeEmbeddings = HuggingFaceBgeEmbeddings(model_name=embed_model_path)

vectordb = Chroma.from_documents(documents=splits, embedding=BgeEmbeddings)

retriever = vectordb.as_retriever()
result = retriever.invoke("美国模式中，政府数据开放战略如何具体推动数据市场的发展？")
result

/home/yhchen/anaconda3/envs/langchain_env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


[Document(metadata={'source': 'data/tencent2.txt'}, page_content='公共数据、基础设施与人工智能：数据市场发展的三个驱动力 | 数据要素行业洞察（四）\n\n王星 腾讯研究院资深专家\n\n根据《数据要素赋能新质生产力》研究报告、中国国际经济交流中心“数据要素推动数字经济高质量发展”专家座谈会发言内容等整理。\n\n二十届三中全会从数据基础设施建设、数据生产关系调整、数据宏观治理优化等维度介绍了数据要素市场的发展导向和政策布局。回顾和总结国内外的发展经验，对于我们准确认识数据要素的发展规律，特别是探索数据要素如何进入社会化大生产、如何安全可控的释放乘数效应价值具有重要借鉴。\n\n“数据二十条”发布近两年来，在政策支持和市场主体的积极参与下，国内数据要素市场呈现出多层次发展和技术驱动的显著特征。对比国外数据市场发展规律，可以看到，以公共数据开放共享为牵引、数据基础设施建设为支撑、人工智能为驱动越来越成为数据要素市场发展的产业共识和重要模式。\n\n一、国外四种典型的数据市场发展模式\n\n1.美国模式：推动政府数据开发利用，鼓励市场主体开展数据应用创新\n\n美国在数据市场构建方面一直处于全球领先位置，率先提出“政府数据开放战略”，以数据驱动政府决策治理能力提升为牵引，大力推动各政府部门探索符合自身特色的数据开发利用场景，改善政府管理数据方式、提高服务社会效能。当前，美国建立了“政府引导、企业参与、市场运作”的数据应用市场，数据要素应用场景发展十分丰富，涵盖消费、农业、医疗、教育、政府管理等多个领域。例如，政府治理领域，美国总务管理局在政府所属基础设施中安装物联网传感器，由美国国家航空航天局利用人工智能技术分析卫星收集的数据，实现智慧建筑管理。Follow My Vote公司开发基于区块链的在线投票平台，采用加密技术保证选举结果的准确性和可靠性。农业领域，纽瓦克垂直农场通过对作物生长环境和长势进行监测，利用大数据技术实现智能决策，相比传统农场节水95%、减肥50%，实现农药零投入。金融领域，华尔街“德温特资本市场”公司通过分析3.4亿账户留言，判断民众情绪，并依据人们高兴时买股票、焦虑时抛售股票的规律，决定公司买卖股票的时机，从而获取盈利。能源领域，提出绿色按钮倡议，使客户能够轻松安全访问用水、电力和天

#### 核心：`create_stuff_documents_chain` 将文档传递给模型

In [3]:
system_prompt = (
    "您是一个用于问答任务的助手。"
    "使用以下检索到的上下文片段来回答问题。"
    "如果您不知道答案，请说您不知道。"
    "最多使用三句话，保持回答简洁。"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# Create a chain for passing a list of Documents to a model.（need `context` key dict as input）
question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

# 构建 retriever chain 需要两个参数， 一个是 retriever， 一个是 combine_docs_chain
rag_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=question_answer_chain,
)

In [4]:
response = rag_chain.invoke({"input": "什么是美国模型？"})
response

{'input': '什么是美国模型？',
 'context': [Document(metadata={'source': 'data/tencent2.txt'}, page_content='公共数据、基础设施与人工智能：数据市场发展的三个驱动力 | 数据要素行业洞察（四）\n\n王星 腾讯研究院资深专家\n\n根据《数据要素赋能新质生产力》研究报告、中国国际经济交流中心“数据要素推动数字经济高质量发展”专家座谈会发言内容等整理。\n\n二十届三中全会从数据基础设施建设、数据生产关系调整、数据宏观治理优化等维度介绍了数据要素市场的发展导向和政策布局。回顾和总结国内外的发展经验，对于我们准确认识数据要素的发展规律，特别是探索数据要素如何进入社会化大生产、如何安全可控的释放乘数效应价值具有重要借鉴。\n\n“数据二十条”发布近两年来，在政策支持和市场主体的积极参与下，国内数据要素市场呈现出多层次发展和技术驱动的显著特征。对比国外数据市场发展规律，可以看到，以公共数据开放共享为牵引、数据基础设施建设为支撑、人工智能为驱动越来越成为数据要素市场发展的产业共识和重要模式。\n\n一、国外四种典型的数据市场发展模式\n\n1.美国模式：推动政府数据开发利用，鼓励市场主体开展数据应用创新\n\n美国在数据市场构建方面一直处于全球领先位置，率先提出“政府数据开放战略”，以数据驱动政府决策治理能力提升为牵引，大力推动各政府部门探索符合自身特色的数据开发利用场景，改善政府管理数据方式、提高服务社会效能。当前，美国建立了“政府引导、企业参与、市场运作”的数据应用市场，数据要素应用场景发展十分丰富，涵盖消费、农业、医疗、教育、政府管理等多个领域。例如，政府治理领域，美国总务管理局在政府所属基础设施中安装物联网传感器，由美国国家航空航天局利用人工智能技术分析卫星收集的数据，实现智慧建筑管理。Follow My Vote公司开发基于区块链的在线投票平台，采用加密技术保证选举结果的准确性和可靠性。农业领域，纽瓦克垂直农场通过对作物生长环境和长势进行监测，利用大数据技术实现智能决策，相比传统农场节水95%、减肥50%，实现农药零投入。金融领域，华尔街“德温特资本市场”公司通过分析3.4亿账户留言，判断民众情绪，并依据人们高兴时买股票、焦虑时抛售股票的规律，决定公司买卖股票的时机，从而获取盈

## Step2. 添加聊天历史

上面的操作是: `query` --> `retriever`;

加入 conversation_history ，则: `(query, conversation_history)` --> `LLM` --> `rephrased query` --> `retriever`


即:

中间多了 `LLM` 处理环节，以及根据 `conversation_history` 重新通过 `LLM` 生成 `new query`, 再通过 `retriever` 获取答案

<br>

---

`create_history_aware_retriever`

>如果没有chat_history ，则输入将直接传递给检索器。如果有chat_history ，则提示和LLM将用于生成搜索查询。然后将该搜索查询传递给检索器。

In [4]:
"""
    这一步是为了构建 `history_aware_retriever`

    这是一个具有 历史聊天内容 感知 的检索器变量(retriever)
"""

from langchain.chains import create_history_aware_retriever     # 用于管理 `chat_history` 为空的情况
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "给定聊天历史和最新的用户问题，"
    "该问题可能引用聊天历史中的上下文，"
    "重新构造一个可以在没有聊天历史的情况下理解的独立问题。"
    "如果需要，不要回答问题，只需重新构造问题并返回。"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

# LLM + (prompt + chat_history) == new hidden query (此步骤不可见)
history_aware_retriever = create_history_aware_retriever(
    llm=llm,
    retriever=retriever,    # retriever = vectordb.as_retriever()
    prompt=contextualize_q_prompt,  # 加入 chat_history 的 new prompt
)

### 核心：将 `history_aware_retriever` 集成到 普通的 `retriever_chain` 中

In [5]:
"""
    普通的 retriever_chain + history_aware_retriever

    即，

    通过普通的 retriever_chain + history_aware_retriever 构建具有历史感知的 chain, 以实现具有历史对话的 RAG
"""

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# 构建 retriever chain 需要两个参数， 一个是 retriever， 一个是 combine_docs_chain
rag_chain = create_retrieval_chain(
    retriever=history_aware_retriever, 
    combine_docs_chain=question_answer_chain,
    )

In [6]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
question = "什么是美国模式？"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1['answer']),
    ]
)

second_question = "那在这个模式中，政府数据开放战略如何具体推动数据市场的发展？"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2['answer'])

在美国模式中，政府数据开放战略通过开放政府数据，鼓励企业和创新者利用这些数据进行应用创新，从而推动数据市场的发展。例如，美国政府推动各政府部门探索特色的数据开发利用场景，改善数据管理方式，提高服务效能，进而促进了数据要素的应用场景多样化，推动了数据市场的繁荣。


In [7]:
print(chat_history)

[HumanMessage(content='什么是美国模式？'), AIMessage(content='美国模式是推动政府数据开发利用，鼓励市场主体开展数据应用创新。通过“政府引导、企业参与、市场运作”的方式，发展数据应用市场，实现数据要素应用场景的丰富化，涵盖多个领域如消费、农业、医疗等。')]


In [9]:
chat_history.extend(
    [
        HumanMessage(content=second_question),
        AIMessage(content=ai_msg_2['answer']),
    ]
)

chat_history

[HumanMessage(content='什么是美国模式？'),
 AIMessage(content='美国模式是指在数据市场构建方面，推动政府数据开发利用，鼓励市场主体开展数据应用创新。美国率先提出“政府数据开放战略”，以数据驱动政府决策治理能力提升为牵引，大力推动各政府部门探索符合自身特色的数据开发利用场景，建立“政府引导、企业参与、市场运作”的数据应用市场。'),
 HumanMessage(content='那在这个模式中，政府数据开放战略如何具体推动数据市场的发展？'),
 AIMessage(content='在这个模式中，政府数据开放战略通过以下方式具体推动数据市场的发展：1. 政府部门开放数据，为市场主体提供丰富的数据资源；2. 鼓励市场主体利用这些开放的数据进行应用创新，促进新服务的开发和现有服务的优化；3. 通过数据驱动提升政府决策治理能力，从而改善政府管理数据方式和提高服务社会效能，为数据市场创造更多需求和应用场景。')]

In [ ]:
?create_retrieval_chain

### 添加历史对话进阶

- 🧐 问题：
前面虽然实现了历史对话，**但仍然是手动更新的**。 

- ⛳️ 目标：
持久化历史对话，实现自动更新历史对话。


<br>


#### 核心两个方法：
- `BaseChatMessageHistory`: 存储对话历史 (持久化)

- `RunnableWithMessageHistory`: 将历史对话注入 input 并在每次调用后更新它



In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory      # 持久化历史对话
from langchain_core.runnables.history import RunnableWithMessageHistory   # 将历史对话注入 input 并在每次调用后更新它

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    runnable=rag_chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [9]:
conversational_rag_chain.invoke(
    {"input": question},
    config={
        "configurable":{"session_id": "1"}
    },
)['answer']

'美国模式是指推动政府数据开发利用，鼓励市场主体开展数据应用创新的发展方式。美国率先提出“政府数据开放战略”，通过改善政府管理数据方式和提高服务社会效能，建立了“政府引导、企业参与、市场运作”的数据应用市场。'

In [11]:
conversational_rag_chain.invoke(
    {"input": second_question},
    config={
        "configurable":{"session_id": "2"}
    }
)['answer']

'政府数据开放战略通过提供公共数据资源，牵引数据市场的发展，鼓励市场主体进行数据应用创新，并推动数据基础设施建设和人工智能技术的融合，形成技术支撑和驱动力。如美国模式所示，政府开放数据促进了各行业领域的数据应用场景丰富，提高了政府管理效率和公共服务能力，同时吸引了企业参与，形成了政府引导、企业参与、市场运作的数据应用市场。'

In [12]:
# 查看历史对话
store

{'1': InMemoryChatMessageHistory(messages=[HumanMessage(content='什么是美国模式？'), AIMessage(content='美国模式是指推动政府数据开发利用，鼓励市场主体开展数据应用创新的发展方式。美国率先提出“政府数据开放战略”，通过改善政府管理数据方式和提高服务社会效能，建立了“政府引导、企业参与、市场运作”的数据应用市场。')]),
 '2': InMemoryChatMessageHistory(messages=[HumanMessage(content='那在这个模式中，政府数据开放战略如何具体推动数据市场的发展？'), AIMessage(content='政府数据开放战略通过提供公共数据资源，牵引数据市场的发展，鼓励市场主体进行数据应用创新，并推动数据基础设施建设和人工智能技术的融合，形成技术支撑和驱动力。如美国模式所示，政府开放数据促进了各行业领域的数据应用场景丰富，提高了政府管理效率和公共服务能力，同时吸引了企业参与，形成了政府引导、企业参与、市场运作的数据应用市场。')])}

## Step3. Agent

将前面构造的 `retriever` 转换为 langchain tools 供 agent 使用

In [15]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retriever, # 比较一下 history_aware_retriever
    name="tencent_post_retriever",
    description="基于腾讯研究院报告内容的检索工具",
)
tools = [retriever_tool]

In [17]:
retriever_tool.invoke({"query": "什么是美国模式？"})

'公共数据、基础设施与人工智能：数据市场发展的三个驱动力 | 数据要素行业洞察（四）\n\n王星 腾讯研究院资深专家\n\n根据《数据要素赋能新质生产力》研究报告、中国国际经济交流中心“数据要素推动数字经济高质量发展”专家座谈会发言内容等整理。\n\n二十届三中全会从数据基础设施建设、数据生产关系调整、数据宏观治理优化等维度介绍了数据要素市场的发展导向和政策布局。回顾和总结国内外的发展经验，对于我们准确认识数据要素的发展规律，特别是探索数据要素如何进入社会化大生产、如何安全可控的释放乘数效应价值具有重要借鉴。\n\n“数据二十条”发布近两年来，在政策支持和市场主体的积极参与下，国内数据要素市场呈现出多层次发展和技术驱动的显著特征。对比国外数据市场发展规律，可以看到，以公共数据开放共享为牵引、数据基础设施建设为支撑、人工智能为驱动越来越成为数据要素市场发展的产业共识和重要模式。\n\n一、国外四种典型的数据市场发展模式\n\n1.美国模式：推动政府数据开发利用，鼓励市场主体开展数据应用创新\n\n美国在数据市场构建方面一直处于全球领先位置，率先提出“政府数据开放战略”，以数据驱动政府决策治理能力提升为牵引，大力推动各政府部门探索符合自身特色的数据开发利用场景，改善政府管理数据方式、提高服务社会效能。当前，美国建立了“政府引导、企业参与、市场运作”的数据应用市场，数据要素应用场景发展十分丰富，涵盖消费、农业、医疗、教育、政府管理等多个领域。例如，政府治理领域，美国总务管理局在政府所属基础设施中安装物联网传感器，由美国国家航空航天局利用人工智能技术分析卫星收集的数据，实现智慧建筑管理。Follow My Vote公司开发基于区块链的在线投票平台，采用加密技术保证选举结果的准确性和可靠性。农业领域，纽瓦克垂直农场通过对作物生长环境和长势进行监测，利用大数据技术实现智能决策，相比传统农场节水95%、减肥50%，实现农药零投入。金融领域，华尔街“德温特资本市场”公司通过分析3.4亿账户留言，判断民众情绪，并依据人们高兴时买股票、焦虑时抛售股票的规律，决定公司买卖股票的时机，从而获取盈利。能源领域，提出绿色按钮倡议，使客户能够轻松安全访问用水、电力和天然气等能源使用数据，提高能源消耗意识，并帮助消费者节省能源消耗。\n\n此外，加大资金布局数字基础设施，为数据要素创新应用发展提供

### 构造 Agent

In [21]:
from langgraph.prebuilt import chat_agent_executor
agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

In [24]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content=question)]}):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='美国模式通常指的是在经济、政治、社会和文化领域中，美国特有的发展模式和实践方式。这个模式强调个人主义、自由市场经济、民主政治和法治等价值观。由于这个问题较为宽泛，我需要调用搜索引擎来获取更详细的信息。<|assistant|>搜索引擎检索\n{"query": "美国模式特点"}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 171, 'total_tokens': 236}, 'model_name': 'glm-4-airx', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-23b05d77-f94c-40a2-b848-10ed1b8af7f1-0', usage_metadata={'input_tokens': 171, 'output_tokens': 65, 'total_tokens': 236})]}}
----


添加 历史聊天记录

In [31]:
# langgraph 内置了持久性，无需 ChatMessageHistory
from langgraph.checkpoint.sqlite import SqliteSaver
with SqliteSaver.from_conn_string(":memory:") as memory:
    agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools, checkpointer=memory)

In [32]:
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm bob")]}, config=config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello, Bob! Nice to meet you. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 172, 'total_tokens': 190}, 'model_name': 'glm-4-airx', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2d9da416-a22d-4d3c-b32c-36bf5f4b84ed-0', usage_metadata={'input_tokens': 172, 'output_tokens': 18, 'total_tokens': 190})]}}
----


In [33]:
temp_question = "什么是欧洲模式？"
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content=temp_question)]}, config=config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='这个问题涉及到一个较为宏观的概念，我需要使用搜索引擎来获取相关信息。<|assistant|>搜索引擎检索\n{"query": "欧洲模式是什么"}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 196, 'total_tokens': 225}, 'model_name': 'glm-4-airx', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ca78e331-d355-4fa3-9a49-ff131b8d171d-0', usage_metadata={'input_tokens': 196, 'output_tokens': 29, 'total_tokens': 225})]}}
----
